In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
attcs = ATTCS()

In [4]:
await attcs.start_task

Read historical data in 1.45 sec
Read historical data in 1.72 sec
Read historical data in 2.46 sec
Read historical data in 2.93 sec
Read historical data in 3.31 sec
Read historical data in 3.91 sec
Read historical data in 6.10 sec
RemoteEvent(ATDome, 0, logMessage) falling behind; read 22 messages


[None, None, None, None, None, None, None]

In [5]:
r = salobj.Remote(attcs.domain, "Script", index=1)

In [6]:
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

In [7]:
await r.start_task

Read historical data in 0.06 sec
Read historical data in 0.22 sec
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 15 messages


In [8]:
await cam.start_task

In [90]:
try:
    await attcs.slew_icrs(ra="09 42 14.4165254340", 
                          dec="-23 54 56.054079074", 
                          rot_pa=-180.-132., 
                          target_name="HD84117")
except asyncio.TimeoutError:
    pass

await attcs.atptg.cmd_offsetAzEl.set_start(az=-10., el=-80., num=0) # zero

Flushing events
Scheduling check coroutines
Sending command
process as completed...
Got False
Telescope not in position


Got True
Telescope in position.


In [54]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=-10., el=-80., num=0) # zero

In [28]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=20., el=-10., num=0) # moving in Y

In [86]:
msg = await r.evt_logMessage.next(flush=True)

In [87]:
print(msg.message)

Grabbed/Wrote extra-focal image 1579762403-HD84117_extra-0-1.fits via wget


In [71]:
msg = await r.evt_logMessage.next(flush=True)

In [72]:
print(msg.message)

Grabbed/Wrote extra-focal image 1579761952-HD84117_extra-0-1.fits via wget


In [89]:
await cam.cmd_startLiveView.set_start(expTime=0.5)

In [ ]:
await asyncio.sleep(10.)

In [91]:
await attcs.atptg.cmd_stopTracking.start()

atptg not in <State.ENABLED: 2>: <State.FAULT: 3>


In [ ]:
7.5*360/24

In [98]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [93]:
await attcs.shutdown()

Disabling ATAOS corrections
ataos not in <State.ENABLED: 2>: <State.FAULT: 3>
Close telescope cover.
Disable ATDomeTrajectory
atdometrajectory not in <State.ENABLED: 2>: <State.DISABLED: 1>
Close dome.


AttributeError: 'NoneType' object has no attribute 'state'

In [95]:
await attcs.atdome.cmd_closeShutter.set_start(timeout=attcs.long_timeout)

AckTimeoutError: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=164284101, ack=<SalRetCode.CMD_INPROGRESS: 301>, error=0, result='')

In [96]:
await attcs.point_azel(target_name="Park position",
                              az=attcs.tel_park_az,
                              el=attcs.tel_park_el,
                              wait_dome=False)

Flushing events
Scheduling check coroutines
Sending command
process as completed...


TimeoutError: 

Got True
Telescope in position.


In [97]:
await attcs.atptg.cmd_stopTracking.start()

atptg not in <State.ENABLED: 2>: <State.FAULT: 3>


In [100]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=attcs.dome_park_az)

In [102]:
set_ss_tasks = []

for comp in attcs.components:
    set_ss_tasks.append(salobj.set_summary_state(attcs._remotes[comp],
                                                 salobj.State.STANDBY,
                                                 timeout=attcs.long_long_timeout))

ret_val = await asyncio.gather(*set_ss_tasks, return_exceptions=True)

print(ret_val)

athexapod not in <State.ENABLED: 2>: <State.DISABLED: 1>
ataos not in <State.ENABLED: 2>: <State.STANDBY: 5>
atdome not in <State.ENABLED: 2>: <State.DISABLED: 1>
atdometrajectory not in <State.ENABLED: 2>: <State.STANDBY: 5>
atmcs not in <State.ENABLED: 2>: <State.DISABLED: 1>
atpneumatics not in <State.ENABLED: 2>: <State.DISABLED: 1>
Task exception was never retrieved
future: <Task finished coro=<ATTCS.wait_for_inposition() done, defined at /home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py:790> exception=TimeoutError()>
Traceback (most recent call last):
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 811, in wait_for_inposition
    status = await asyncio.gather(*wait_tasks)
  File "/home/saluser/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py", line 862, in wait_for_atdome_inposition
    in_position = await self.atdome.evt_azimuthInPosition.aget(timeout=timeout)
  File "